# NX-421 Mini-Project 1 — Part 1 (Step 1 & 2 )

In [2]:
%gui wx
import sys
import os

notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

os.environ["DIPY_HOME"] = "/data"

import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

loadFSL()

import fsl.wrappers
from fsl.wrappers import fslmaths, epi_reg , mcflirt
import mne
import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri


import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report

import requests
import urllib.request
from tqdm import tqdm
from fsl.wrappers import fast, bet , flirt 
from fsl.wrappers.misc import fslroi
import glob
import pandas as pd
import numpy as np
import json
import subprocess
import matplotlib.pyplot as plt

Gtk-Message: 15:26:07.021: Failed to load module "canberra-gtk-module"


In [3]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

15:28:28: Debug: Adding duplicate image handler for 'Windows bitmap file'
15:28:28: Debug: Adding duplicate animation handler for '1' type
15:28:28: Debug: Adding duplicate animation handler for '2' type
15:28:28: Debug: Adding duplicate image handler for 'Windows bitmap file'
15:28:28: Debug: Adding duplicate animation handler for '1' type
15:28:28: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:2023): Gtk-CRITICAL **: 15:28:28.527: gtk_window_resize: assertion 'height > 0' failed


## Structural Preprocessing 

### Loading structural data 

In [ ]:
fsleyesDisplay.load(op.expandvars('T1w.nii'))

In [7]:
BASE_DATA_ROOT = "/data"
mkdir_no_exist(BASE_DATA_ROOT)
deriv_= op.join(BASE_DATA_ROOT, 'derivatives')

Defining preprocessed path 

In [8]:
PREPROCESS_DATA_ROOT = op.join(BASE_DATA_ROOT, 'derivatives','preprocessed_data')
mkdir_no_exist(PREPROCESS_DATA_ROOT)

### Skull Stripping with `BET`

In [7]:
def skull_stripped(sample_root, preproc_root, robust=False):
    """
    Perform skull stripping on a T1w anatomical image and save directly in preproc_root.
    
    Args:
        sample_root (str): Path to the directory containing the input T1w image.
        preproc_root (str): Path to save the skull-stripped outputs.
        robust (bool): Whether to use robust mode (-R) in BET.
    """
    anatomical_path = op.join(sample_root, 'T1w.nii.gz')
    betted_brain_path = op.join(preproc_root, 'T1w_stripped')
    os.system(f'bet {anatomical_path} {betted_brain_path} -m {"-R" if robust else ""}')
    print("Done with BET.")

In [8]:
skull_stripped(BASE_DATA_ROOT, PREPROCESS_DATA_ROOT)

Done with BET.


In [10]:
resulting_mask_path = op.join(PREPROCESS_DATA_ROOT, 'T1w_stripped_mask.nii.gz')

In [11]:
fsleyesDisplay.load(resulting_mask_path)

Setting Robust = TRUE 

In [14]:
skull_stripped(BASE_DATA_ROOT, PREPROCESS_DATA_ROOT, robust=True)

Done with BET.


In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.join(BASE_DATA_ROOT, 'T1w.nii'))
fsleyesDisplay.load(resulting_mask_path)

### Tissue segmentation with `FAST`

In [17]:
anatomical_path = op.join(BASE_DATA_ROOT, 'T1w.nii.gz')
bet_path = op.join(PREPROCESS_DATA_ROOT,'T1w_stripped.nii.gz') 

In [19]:
fast_target = bet_path 
[os.remove(f) for f in glob.glob(op.join(PREPROCESS_DATA_ROOT, '*fast*'))] 
segmentation_path = op.join(PREPROCESS_DATA_ROOT, 'T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

{}

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(glob.glob(op.join(PREPROCESS_DATA_ROOT, '*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(PREPROCESS_DATA_ROOT, '*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(PREPROCESS_DATA_ROOT, '*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## Functional Preprocessing 

### Loading fMRI

In [25]:
run1 = nib.load("tfMRI_MOTOR_LR.nii")
run2 = nib.load("tfMRI_MOTOR_RL.nii")

In [27]:
run1= "tfMRI_MOTOR_LR.nii"
run2= "tfMRI_MOTOR_RL.nii"

Rescaling variance to 1

In [28]:
# Get the global standard deviation (SD) from fslstats
def get_sd(file):
    sd = os.popen(f"fslstats {file} -V").read().split()[1]
    return float(sd)

# Get SD for both runs
sd_run1 = get_sd(run1)
sd_run2 = get_sd(run2)

# Rescale each file using fslmaths
os.system(f"fslmaths {run1} -div {sd_run1} {run1.replace('.nii', '_rescaled.nii')}")
os.system(f"fslmaths {run2} -div {sd_run2} {run2.replace('.nii', '_rescaled.nii')}")

0

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(op.expandvars('tfMRI_MOTOR_LR_rescaled.nii.gz'))

### Concatenation 

In [ ]:
# Concatenate the 4D files
os.system("fslmerge -t concatenated_rescaled.nii.gz tfMRI_MOTOR_LR_rescaled.nii.gz tfMRI_MOTOR_RL_rescaled.nii.gz")

In [ ]:
fsleyesDisplay.load(op.expandvars('concatenated_rescaled.nii.gz'))

In [ ]:
functional_data_path = 'concatenated_rescaled.nii.gz'
moco_data_path = 'concatenated_rescaled_moco.nii.gz'

### Motion correction 

In [ ]:
mcflirt(infile=functional_data_path,
        o=moco_data_path,
        plots=True,
        report=True,
        dof=6,
        mats=True)

In [ ]:
def load_mot_params_fsl_6_dof(path):
    return pd.read_csv(path, sep='  ', header=None, 
            engine='python', names=['Rotation x', 'Rotation y', 'Rotation z','Translation x', 'Translation y', 'Translation z'])

In [ ]:
mot_params = load_mot_params_fsl_6_dof('concatenated_rescaled_moco.nii.gz.par')
mot_params


In [ ]:
def compute_FD_power(mot_params):
    framewise_diff = mot_params.diff().iloc[1:]

    rot_params = framewise_diff[['Rotation x', 'Rotation y', 'Rotation z']]
    converted_rots = rot_params*50
    trans_params = framewise_diff[['Translation x', 'Translation y', 'Translation z']]
    fd = converted_rots.abs().sum(axis=1) + trans_params.abs().sum(axis=1)
    return fd

fd = compute_FD_power(mot_params).to_numpy()

threshold = np.quantile(fd,0.75) + 1.5*(np.quantile(fd,0.75) - np.quantile(fd,0.25))

plt.plot(list(range(1, fd.size+1)), fd)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
plt.hlines(threshold, 0, 284,colors='black', linestyles='dashed', label='FD threshold')
plt.legend()
plt.show()



In [ ]:
functional_data_path_LR = 'tfMRI_MOTOR_LR_rescaled.nii.gz'
moco_data_path_LR = 'tfMRI_MOTOR_LR_rescaled_moco.nii.gz'

functional_data_path_RL = 'tfMRI_MOTOR_RL_rescaled.nii.gz'
moco_data_path_RL = 'tfMRI_MOTOR_RL_rescaled_moco.nii.gz'


In [ ]:
mcflirt(infile=functional_data_path_LR,
        o=moco_data_path_LR,
        plots=True,
        report=True,
        dof=6,
        mats=True)

mcflirt(infile=functional_data_path_RL,
        o=moco_data_path_RL,
        plots=True,
        report=True,
        dof=6,
        mats=True)


In [ ]:
mot_params_LR = load_mot_params_fsl_6_dof('tfMRI_MOTOR_LR_rescaled_moco.nii.gz.par')
mot_params

mot_params_RL = load_mot_params_fsl_6_dof('tfMRI_MOTOR_RL_rescaled_moco.nii.gz.par')
mot_params

In [ ]:
fd_RL = compute_FD_power(mot_params_RL).to_numpy()

threshold_RL = np.quantile(fd_RL,0.75) + 1.5*(np.quantile(fd_RL,0.75) - np.quantile(fd_R,0.25))

import matplotlib.pyplot as plt

#%matplotlib inline
plt.plot(list(range(1, fd.size+1)), fd)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
plt.hlines(threshold, 0, 284,colors='black', linestyles='dashed', label='FD threshold')
plt.legend()
plt.show()

### Co-registration using `epi_reg` 

In [5]:
subprocess.run(['epi_reg'])


Usage: epi_reg [options] --epi=<EPI image> --t1=<wholehead T1 image> --t1brain=<brain extracted T1 image> --out=<output name>
 
Optional arguments
  --fmap=<image>         : fieldmap image (in rad/s)
  --fmapmag=<image>      : fieldmap magnitude image - wholehead extracted
  --fmapmagbrain=<image> : fieldmap magnitude image - brain extracted
  --gdc=<image>          : Gradient-distortion corection warpfield
  --wmseg=<image>        : white matter segmentation of T1 image
  --echospacing=<val>    : Effective EPI echo spacing (sometimes called dwell time) - in seconds
  --pedir=<dir>          : phase encoding direction, dir = x/y/z/-x/-y/-z
  --weight=<image>       : weighting image (in T1 space)
  --nofmapreg            : do not perform registration of fmap to T1 (use if fmap already registered) 
  --noclean              : do not clean up intermediate files
  -v                     : verbose output
  -h                     : display this help message
 
e.g.:  epi_reg --epi=example_func

CompletedProcess(args=['epi_reg'], returncode=1)

For co-registration , we extract using fslroi the scan 250, which is a middle and stable scan, according to the framewise displacement plot. It also shows a good contrast between WM & GM. 
We use the WM segmentation from FAST for a more accurate alignment in terms of contrast between the two modalities, functional and anatomical.

In [ ]:
epi_source = op.join("concatenated_rescaled_moco.nii.gz" )
whole_t1 = op.join("T1w.nii.gz")
skull_stripped_t1 = op.join(PREPROCESS_DATA_ROOT,"T1w_stripped.nii")
output_path = op.join(PREPROCESS_DATA_ROOT, 'func_struct_coregis')
ref_vol_name =  op.join(PREPROCESS_DATA_ROOT,'bold_moco_vol_middle')

use_single_vol = True

if use_single_vol:
    # Extracting a stable middle volume with fslroi 
    fslroi(epi_source, ref_vol_name, str(250), str(1))

    subprocess.run(['epi_reg','--epi={}'.format(ref_vol_name), '--t1={}'.format(whole_t1), '--t1brain={}'.format(skull_stripped_t1), '--out={}'.format(output_path)])
else:
    subprocess.run(['epi_reg','--epi={}'.format(epi_source), '--t1={}'.format(whole_t1), '--t1brain={}'.format(skull_stripped_t1), '--out={}'.format(output_path)])

white_matter_segmentation = op.join(PREPROCESS_DATA_ROOT, "T1w_fast_pve_2.nii.gz") 

subprocess.run(['epi_reg','--epi={}'.format(ref_vol_name), '--t1={}'.format(whole_t1), '--t1brain={}'.format(skull_stripped_t1), 
                '--out={}'.format(output_path),
               '--wmseg={}'.format(white_matter_segmentation)])

Overlaying EPI & anatomical & Visualizing 

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(skull_stripped_t1)
fsleyesDisplay.load(output_path)

### Gaussian smoothing

In [ ]:
input_path = 'concatenated_rescaled_moco.nii.gz'

In [ ]:
cmd = 'fslmaths {} -s {} {}_smoothed-6mm'.format(input_path, 6/2.3548, input_path)
subprocess.run(['fslmaths',input_path, '-s', str(6/2.3548), '{}_smoothed-6mm'.format(input_path)])

In [ ]:
fsleyesDisplay.load(input_path + '_smoothed-6mm')